# 🐡 The sharp points of Rockpool 🐚

In [ ]:
# - Rockpool imports
from rockpool import TSContinuous

# - General imports and configuration
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 4]
plt.rcParams['figure.dpi'] = 300

## 🦀 How to use sampled time series data (in)correctly in Rockpool
Time series data loaded from elsewhere probably comes in a clocked raster format. You can easily use this data in Rockpool, but there are a couple of tricky points to watch out for.

### Wrong: how to generate a time base for clocked data

In [ ]:
T = 1000
dt = 1e-3
data = np.random.rand(T)
t_start = 23.6
time_base = np.arange(t_start, t_start + len(data) * dt, dt)

In [ ]:
time_base = np.arange(T) * dt + t_start

### Wrong: how to define a time series from clocked data

Say we have a 20-second sample, sampled on a 100ms clock. Let's convert this into a continuous time series:

In [ ]:
dt = 100e-3
T = round(20 / dt)
data = np.random.rand(T)
time_base = np.arange(len(data)) * dt

ts = TSContinuous(time_base, data)
ts

In [ ]:
ts = TSContinuous(time_base, data, t_stop = 20.)
ts

In [ ]:
ts = TSContinuous.from_clocked(data, dt=dt)
ts

## 🐚 Defining extents for `TSEvent` time series data

In [ ]:
from rockpool import TSEvent
from matplotlib import pyplot as plt

times = [.2, .8, 1.2, 1.4, 1.8, 2.2, 3.3, 3.5, 3.6, 4.2, 4.8, 5.2, 5.8, 6.2, 6.5, 6.8]
channels = [0, 3, 3, 1, 6, 6, 3, 3, 5, 5, 4, 0, 5, 1, 1, 2]
ts = TSEvent(times, channels, t_start = 0., t_stop = 7.)
ts.plot();

In [ ]:
from IPython.display import Image
Image("TSEvent_to_raster.png")

```python
def raster(
    dt: float,
    t_start: float=None,
    t_stop: float=None,
    num_timesteps: int=None,
    channels: numpy.ndarray=None,
    add_events: bool=False,
    include_t_stop: bool=False,
) -> numpy.ndarray:
```

In [ ]:
tsBad = TSEvent(times, channels, t_stop=7., name = 'No extents 🤢🤬')
print(tsBad)

tsGood = TSEvent(times, channels, t_start = 0., t_stop = 7., name = 'With extents 😇🥰')
print(tsGood)

## 🐡 `TimeSeries` and `Layer`s share an explicit global time base

In [ ]:
# - Imports
from rockpool.layers import FFRateEuler
from rockpool import TSContinuous

# - Define a layer with a single rate neuron
lyr = FFRateEuler([1], dt = 1e-3)
print('lyr:', lyr)

# - Define a time series
data = np.sin(np.arange(0, 10, dt) / 4 * (2*np.pi))
ts_input = TSContinuous.from_clocked(data, dt = .1)
print('ts_input:', ts_input)

Now we evolve `lyr` using `ts_input`, and look at the result.

In [ ]:
lyr.evolve(ts_input).plot();

But if we try to evolve `lyr` again in the same way, we recieve an error.

In [ ]:
lyr.evolve(ts_input)

In [ ]:
ts_input = TSContinuous.from_clocked(data, dt = .1, t_start = lyr.t)
print('ts_input:', ts_input)
lyr.evolve(ts_input)

In [ ]:
# - Using `.start_at()`
lyr.evolve(ts_input.start_at(lyr.t))

# - Using `.delay()`
lyr.evolve(ts_input.delay(lyr.t - ts_input.t_start))

In [ ]:
ts_input = TSContinuous.from_clocked(data, dt = .1, periodic = True)
ts_input.plot(np.arange(-10, 20, .1));

In [ ]:
lyr.evolve(ts_input)
lyr.evolve(ts_input)

## 🦀 `Layer`s in Rockpool are dynamical systems, and don't get reset implicitly